# ================================================
# 1. HuggingFace Hub에 Model 등록
# ================================================

In [1]:
import os
from huggingface_hub import HfApi
from huggingface_hub.utils import HfHubHTTPError
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

In [ ]:
# 0) Hugging Face 토큰을 환경변수에서 불러오기

# token = os.environ["HF_TOKEN"]                    # 이미 HF_TOKEN 환경변수에 토큰이 설정되어 있다고 가정
token = "MY_HF_TOKEN"                               # 또는 토큰 직접 넣어주기
repo_id = "SIQRIT/DAIS-Qwen3-8B-qdora"              # HuggingFace Hub의 PEFT 체크포인트 경로
peft_dir = "./dais_model/"                          # 로컬 저장소 경로

In [3]:
# 1) 허브에 저장소(repository) 생성 (이미 있다면무시)

api = HfApi()

try:
    api.create_repo(repo_id=repo_id, token=token, private=False)
    print(f"✅ 저장소 '{repo_id}' 생성 완료.")
except HfHubHTTPError:
    # 409 Conflict 등 모든 HfHubHTTPError는 기존 리포지토리가 이미 있는 경우로 간주
    print(f"ℹ️ 저장소 '{repo_id}'가 이미 존재합니다. 계속 진행합니다.")

ℹ️ 저장소 'SIQRIT/DAIS-Qwen3-8B-qdora'가 이미 존재합니다. 계속 진행합니다.


In [4]:
# 2) 베이스 모델 로드 (Qwen/Qwen3-8B)

base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-8B",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
# 3) 로컬 PEFT 어댑터 로드

peft_model = PeftModel.from_pretrained(
    base_model,    # 위치 인자 #1: 베이스 모델 인스턴스
    peft_dir,      # 위치 인자 #2: PEFT 체크포인트 디렉토리
    ignore_mismatched_sizes=True,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:470: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([151677, 4096]) in the checkpoint and torch.Size([151936, 4096]) in the model instantiated
- base_model.model.model.embed_tokens.base_layer.weight: found shape torch.Size([151677, 4096]) in the checkpoint and torch.Size([151936, 4096]) in the model instantiated
- base_model.model.model.embed_tokens.lora_embedding_A.default: found shape torch.Size([64, 151677]) in the checkpoint and torch.Size([64, 151936]) in the model instantiated.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.embed_tokens.lora_embedding_A.default'].
  warnings.warn(warn_message)


In [6]:
# 4) 토크나이저 로드 (스페셜 토큰 포함된 버전)

tokenizer = AutoTokenizer.from_pretrained(
    peft_dir,
    trust_remote_code=True,
    use_fast=True
)

In [7]:
# 5) 허브에 모델과 토크나이저 업로드

peft_model.push_to_hub(repo_id, use_auth_token=token)
tokenizer.push_to_hub(repo_id, use_auth_token=token)

print(f"✅ 모델과 토크나이저를 '{repo_id}'에 성공적으로 업로드했습니다.")

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:907: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/744M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✅ 모델과 토크나이저를 'SIQRIT/DAIS-Qwen3-8B-qdora'에 성공적으로 업로드했습니다.


# ================================================
# 2. 공개 저장소로 전환
# ================================================

In [8]:
from huggingface_hub import HfApi
from huggingface_hub.utils import HfHubHTTPError

api = HfApi()

try:
    api.update_repo_visibility(
        repo_id=repo_id,
        private=False,        # ← 공개로 변경
        token=token
    )
    print(f"✅ 리포지토리 '{repo_id}'를 공개로 전환했습니다.")
except HfHubHTTPError as e:
    print("❌ 공개 전환 중 오류 발생:", e)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'update_repo_visibility' (from 'huggingface_hub.hf_api') is deprecated and will be removed from version '0.32'. Please use `update_repo_settings` instead.
  warnings.warn(warning_message, FutureWarning)


✅ 리포지토리 'SIQRIT/DAIS-Qwen3-8B-qdora'를 공개로 전환했습니다.
